In [1]:
import os

# Install required packages
%pip install scrapy
%pip install selenium
%pip install pandas
%pip install crochet
%pip install scrapy-splash
%pip install scrapy playwright
# %scrapy crawl searcher

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from datetime import datetime
from typing import Iterable
import scrapy
from scrapy_selenium import SeleniumRequest
from scrapy_splash import SplashRequest

class ItemFormat(scrapy.Item):
    card_name = scrapy.Field()
    # percentage_in_decks = scrapy.Field()
    # img = scrapy.Field()
    # specials = scrapy.Field()
    # price_per_kg = scrapy.Field()

class knightlySpider(scrapy.Spider):
    name = 'searcher'

    custom_settings = {
        'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)',
        'FEED_FORMAT': 'json',
        'FEED_URI': f'knightlyScrape_{datetime.now().strftime("%Y%m%d%H%M%S")}.json',
        'SPLASH_URL': 'http://localhost:8050',
    }

    def start_requests(self) -> Iterable[scrapy.Request]:
        urls = ['https://www.knightlygaming.co.za/collections/yugioh-singles?page=1']

        for url in urls:
            yield scrapy.Request(url=url, meta={'playwright': True}, callback=self.parse)
        return super().start_requests()

    def parse(self, response):

        next_page_link = response.css("div.pag_next::attr(href)").get()
        if next_page_link:
            yield scrapy.Request(url=next_page_link, meta={'playwright': True}, callback=self.parse)

        for i, cards in enumerate(response.css('li.grid__item.col-lg-4.col-md-6.col-sm-6.col-xs-6')):
            cards_item = ItemFormat()

            cards_item['card_name'] = cards.css('div.h4.grid-view-item__title::text').get(i)
            # cards_item['percentage_in_decks'] = cards.css('div.CardLabel_label__iAM7T::text').get().replace("\n", " || synergy ")
            # quote_item['img'] = quote.css('a img.CardImage_border__OcVcj.shadow::attr(src)').get()

            yield cards_item

In [3]:
from scrapy.crawler import CrawlerProcess


process = CrawlerProcess({
})

process.crawl(knightlySpider)
process.start()

2024-06-04 01:38:01 [scrapy.utils.log] INFO: Scrapy 2.11.1 started (bot: scrapybot)
2024-06-04 01:38:01 [scrapy.utils.log] INFO: Versions: lxml 5.2.1.0, libxml2 2.11.7, cssselect 1.2.0, parsel 1.9.1, w3lib 2.1.2, Twisted 24.3.0, Python 3.12.0 (tags/v3.12.0:0fb18b0, Oct  2 2023, 13:03:39) [MSC v.1935 64 bit (AMD64)], pyOpenSSL 24.1.0 (OpenSSL 3.2.1 30 Jan 2024), cryptography 42.0.7, Platform Windows-10-10.0.19045-SP0
2024-06-04 01:38:01 [scrapy.addons] INFO: Enabled addons:
[]
2024-06-04 01:38:01 [py.warnings] WARNING: c:\Users\josia\AppData\Local\Programs\Python\Python312\Lib\site-packages\scrapy\utils\request.py:254: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward compatibility reasons, but it will change in a future version of Scrapy.